# Hubbard Model: A Tunable Model of Electron Correlation
**Evangeslista Group Rotation Project**

Brian Zhao, 10 Oct 2022

We first consider the simplest case, an open, one-dimensional Hubbard model. For an $N$-site $1$-D Hubbard model, we represent a configuration as
$$
|\mathbf{\text{n}}\rangle = |\underbrace{k_{1\uparrow}k_{2\uparrow}k_{3\uparrow}\dots k_{N\uparrow}}_{\text{alpha}}\underbrace{k_{1\downarrow}k_{2\downarrow}k_{3\downarrow}\dots k_{N\downarrow}}_{\text{beta}}\rangle
$$
where $k_i$ is the occupatation number of the $i$-th spinorbital.

In [53]:
import numpy as np
import scipy.special

class HubbardLattice1D:
    def __init__(self, nsites, nalpha, nbeta, t, u):
        self.nsites = nsites
        self.nalpha = nalpha
        self.nbeta = nbeta
        self.t = t
        self.u = u
        self.ms = (nalpha-nbeta)/2.0
        
        # Unlikely do be doing more than 32 sites for now..
        #self.bit_string_len = int(np.ceil(self.nsites*2/64))

def enumerate_states(hl):
    """
    Enumerates all states of hl, only conserving Ms symmetry.
    """
    nalpha_combinations = int(scipy.special.binom(hl.nsites, hl.nalpha))
    nbeta_combinations = int(scipy.special.binom(hl.nsites, hl.nbeta))
    ncomb = nalpha_combinations * nbeta_combinations
    mem_reqd = ncomb*8/1e3 # We use 64-bit integers
    
    print(f"""There are {nalpha_combinations:d} alpha combinations,
{nbeta_combinations:d} beta combinations,
making a total of {ncomb:d} combinations,
requiring a memory of {mem_reqd:.2f} kB.""")
    
    alpha_strings = np.zeros(nalpha_combinations, dtype='int64')
    beta_strings = np.zeros(nbeta_combinations, dtype='int64')
    tot_strings = np.zeros(ncomb, dtype='int64')
    
    comb = list(range(hl.nalpha))
    alpha_strings[0] = occ_list_to_bit_string(comb)
    for i in range(1,nalpha_combinations):
        ierr, comb = next_comb(hl.nsites, hl.nalpha, comb)
        if (ierr == 1):
            print('Too many combinations! Breaking.')
            break
        alpha_strings[i] = occ_list_to_bit_string(comb)
    
    comb = list(range(hl.nbeta))
    beta_strings[0] = occ_list_to_bit_string(comb)
    for i in range(1,nbeta_combinations):
        ierr, comb = next_comb(hl.nsites, hl.nbeta, comb)
        if (ierr == 1):
            print('Too many combinations! Breaking.')
            break
        beta_strings[i] = occ_list_to_bit_string(comb)
    
    istring = 0
    for ialpha in range(nalpha_combinations):
        for ibeta in range(nbeta_combinations):
            tot_strings[istring] = ((beta_strings[ibeta])<<hl.nbeta) | (alpha_strings[ialpha])
            istring += 1
    
    return alpha_strings, beta_strings, tot_strings

    
def next_comb(n, k, comb):
    i = k-1
    comb[i] += 1
    
    while (comb[i] >= n - k + 1 + i):
        i -= 1
        if (i < 0):
            break
        comb[i] += 1
    
    if (comb[0] > n-k):
        ierr = 1
    else:
        ierr = 0
        for j in range(i+1, k):
            comb[j] = comb[j-1] + 1
            
    return ierr, comb

def occ_list_to_bit_string(occ_list):
    bstring = 0
    for i in range(len(occ_list)):
        bstring = bstring | (1<<occ_list[i])
    return bstring

## Testing ground below

In [47]:
occ_list_to_bit_string([0,1,2,4])

23

In [54]:
hl = HubbardLattice1D(4, 2, 2, 1, 0)
enumerate_states(hl)

There are 6 alpha combinations,
6 beta combinations,
making a total of 36 combinations,
requiring a memory of 0.29 kB.


(array([ 3,  5,  9,  6, 10, 12]),
 array([ 3,  5,  9,  6, 10, 12]),
 array([15, 23, 39, 27, 43, 51, 13, 21, 37, 29, 45, 53, 13, 29, 45, 25, 41,
        57, 14, 22, 38, 30, 46, 54, 14, 30, 46, 26, 42, 58, 12, 28, 44, 28,
        44, 60]))

In [31]:
comb = list(range(4))
print(comb)
ncomb = 1
while True:
    ierr,comb = next_comb(8,4,comb)
    if (ierr == 1):
        break
    print(comb)
    ncomb += 1
print(f'ncomb = {ncomb}')

[0, 1, 2, 3]
[0, 1, 2, 4]
[0, 1, 2, 5]
[0, 1, 2, 6]
[0, 1, 2, 7]
[0, 1, 3, 4]
[0, 1, 3, 5]
[0, 1, 3, 6]
[0, 1, 3, 7]
[0, 1, 4, 5]
[0, 1, 4, 6]
[0, 1, 4, 7]
[0, 1, 5, 6]
[0, 1, 5, 7]
[0, 1, 6, 7]
[0, 2, 3, 4]
[0, 2, 3, 5]
[0, 2, 3, 6]
[0, 2, 3, 7]
[0, 2, 4, 5]
[0, 2, 4, 6]
[0, 2, 4, 7]
[0, 2, 5, 6]
[0, 2, 5, 7]
[0, 2, 6, 7]
[0, 3, 4, 5]
[0, 3, 4, 6]
[0, 3, 4, 7]
[0, 3, 5, 6]
[0, 3, 5, 7]
[0, 3, 6, 7]
[0, 4, 5, 6]
[0, 4, 5, 7]
[0, 4, 6, 7]
[0, 5, 6, 7]
[1, 2, 3, 4]
[1, 2, 3, 5]
[1, 2, 3, 6]
[1, 2, 3, 7]
[1, 2, 4, 5]
[1, 2, 4, 6]
[1, 2, 4, 7]
[1, 2, 5, 6]
[1, 2, 5, 7]
[1, 2, 6, 7]
[1, 3, 4, 5]
[1, 3, 4, 6]
[1, 3, 4, 7]
[1, 3, 5, 6]
[1, 3, 5, 7]
[1, 3, 6, 7]
[1, 4, 5, 6]
[1, 4, 5, 7]
[1, 4, 6, 7]
[1, 5, 6, 7]
[2, 3, 4, 5]
[2, 3, 4, 6]
[2, 3, 4, 7]
[2, 3, 5, 6]
[2, 3, 5, 7]
[2, 3, 6, 7]
[2, 4, 5, 6]
[2, 4, 5, 7]
[2, 4, 6, 7]
[2, 5, 6, 7]
[3, 4, 5, 6]
[3, 4, 5, 7]
[3, 4, 6, 7]
[3, 5, 6, 7]
[4, 5, 6, 7]
ncomb = 70


In [28]:
next_comb(8,4,comb)

(0, [0, 1, 4, 6])